# Retrieve sensor log from Raspberry Pi and graph it
This version works with monthly created csv files *_yyyy-mm.csv

In [ ]:
# Read raspberry pi access data from secrets file

import sys
sys.path.append('../AlvaziLib')

from Secrets import Secrets
PATH = "../Secrets/"
SECRETS_FILENAME = "AlvaziSecrets.json"
secrets = Secrets(PATH, SECRETS_FILENAME)

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import glob

# Set log file name pattern: sensor_log_yyyy-mm.csv
LogFilePattern = 'sensor_log_{}-{}.csv'

# get list of local files that match this pattern
csv_files = glob.glob(LogFilePattern.format('*', '*'))
print(csv_files)

# Note: renamed the original big file to sensor_log_2023-00 to fit into the pattern

In [ ]:
import datetime
import pytz

# set my time zone
myTimeZone = 'US/Pacific'

#get current year and month
now = datetime.datetime.now(pytz.timezone(myTimeZone))
year = now.strftime("%Y")
month = now.strftime("%m")

In [ ]:
from paramiko import SSHClient
from scp import SCPClient

# get login info for my Raspberry Pi
RaspberryPi = secrets.secret("Raspberry Pi 2013")["IP"]
MyUser = secrets.secret("Raspberry Pi 2013")["USER"]
MyPassword = secrets.secret("Raspberry Pi 2013")["PASSWORD"]
RemotePath = 'Adafruit_Python_BMP/'

# add sensor log file for current month (for the case when not yet transferred)
LogFile = LogFilePattern.format(year, month)
if LogFile not in csv_files:
    csv_files.append(LogFile)

# get ssh client to connect to my Raspberry Pi
ssh = SSHClient() #secure shell SSH connection
ssh.load_system_host_keys()
ssh.connect(RaspberryPi, username=MyUser, password=MyPassword, look_for_keys=False)
scp = SCPClient(ssh.get_transport())

# copy sensor log files from my Raspberry Pi
# only get the two latest monthly files (assuming that last download is not more than 2 months ago)
csv_files.sort(reverse=True)
for csv_file in csv_files[:2]:
    if csv_file != 'sensor_log_2023-00.csv':
        print(csv_file)
        scp.get(RemotePath+csv_file)
scp.close()

In [ ]:
import pandas as pd

# read sensor log files (CSV format) into pandas dataframe objects
# concatenate into a single dataframe object
frames = []
csv_files.sort(reverse=False)
for csv_file in csv_files:
    print(csv_file)
    df = pd.read_csv(csv_file, index_col=0, parse_dates=True, usecols=['TimeStamp', 'Temp', 'Pressure'])
    frames.append(df)
sensorLog = pd.concat(frames)

# round timestamp to full minutes
sensorLog.index = pd.to_datetime(sensorLog.index.map(lambda t: t.strftime('%Y-%m-%d %H:%M')))

# localize timestamp index to my timezone
sensorLog.index = sensorLog.index.tz_localize(myTimeZone, ambiguous='infer')

# print most recent 10 sensor log entries
sensorLog.tail(10)

In [ ]:
# timestamp string for plots
UpdatedTimestamp = 'Updated {}'.format(now.strftime('%Y-%m-%d %H:%M'))

In [ ]:
import matplotlib.pyplot as plt

# plot all data columns
axs = sensorLog.plot(figsize=(20,20), grid=True, subplots=True)

In [ ]:
today = datetime.date.today().isoformat()
print(today)

# plot today's temperature and pressure
axs = sensorLog.loc[today:today].plot(y=['Temp','Pressure'], figsize=(20,10), grid=True, marker='o', subplots=True)

print(sensorLog.loc[today:today].describe())

In [ ]:
# plot yesterday's and today's temperature and pressure
yesterday = (datetime.date.today() - datetime.timedelta(days = 1)).isoformat()
axs = sensorLog.loc[yesterday:today].plot(y=['Temp','Pressure'], figsize=(20,10), grid=True, marker='.', subplots=True)

In [ ]:
# plot past week's temperature and pressure
weekago = (datetime.date.today() - datetime.timedelta(weeks = 1)).isoformat() 
axs = sensorLog.loc[weekago:today].plot(y=['Temp','Pressure'], figsize=(20,10), grid=True, subplots=True, title=f'Past Week / {UpdatedTimestamp}')

# save plot as file
plt.savefig('pastWeekPlot')

In [ ]:
sensorLog.axes

In [ ]:
sensorLog.loc[weekago:today].agg(['min', 'max', 'mean'])

In [ ]:
# 1-day rolling average
axs = sensorLog.rolling(24*4).mean().plot(y=['Temp','Pressure'], figsize=(20,10), grid=True, subplots=True, title=f'1-day rolling average / {UpdatedTimestamp}')
# save plot as file
plt.savefig('rollingAverage-1day')

In [ ]:
# 3 days rolling average
axs = sensorLog.rolling(3*24*4).mean().plot(y=['Temp','Pressure'], figsize=(20,10), grid=True, subplots=True, title=f'3-days rolling average / {UpdatedTimestamp}')

In [ ]:
# 7 days rolling average
axs = sensorLog.rolling(7*24*4).mean().plot(y=['Temp','Pressure'], figsize=(20,10), grid=True, subplots=True, title=f'7-days rolling average / {UpdatedTimestamp}')
# save plot as file
plt.savefig('rollingAverage-7days')

In [ ]:
sensorLogDiff = sensorLog.diff()
axs = sensorLogDiff.loc[weekago:today].rolling(4*4).mean().plot(figsize=(20,20), grid=True, subplots=True, title=f'Difference 4-hour rolling average / {UpdatedTimestamp}')
plt.savefig('Diff-rollingAverage-4hours')

In [ ]:
# bar plot daily average, minimum, and maximum temperature

axs = sensorLog.resample('D', level=0).mean().plot.bar(title='Daily Average Temperature',
    y=['Temp'], figsize=(20,10), grid=True)
axs1 = sensorLog.resample('D', level=0).min().plot.bar(
    y=['Temp'], figsize=(20,10), grid=True, title='Daily Minimum Temperature')
axs2 = sensorLog.resample('D', level=0).max().plot.bar(
    y=['Temp'], figsize=(20,10), grid=True, title='Daily Maximum Temperature')

In [ ]:
# calculate daily min, mean, max temperature using dataframe resampling
minTemp = sensorLog.loc[:yesterday].resample('D', level=0).min().filter(items=['TimeStamp','Temp']).rename(columns = {'Temp':'Min'})
meanTemp = sensorLog.loc[:yesterday].resample('D', level=0).mean().filter(items=['TimeStamp','Temp']).rename(columns = {'Temp':'Mean'})
maxTemp = sensorLog.loc[:yesterday].resample('D', level=0).max().filter(items=['TimeStamp','Temp']).rename(columns = {'Temp':'Max'})
dailyTemp = minTemp.join(meanTemp, how='outer').join(maxTemp, how='outer')
axs = dailyTemp.plot(figsize=(20,5), grid=True, subplots=False, marker='.', title='Daily Temperature [C]')

In [ ]:
# Calculate and plot Fourier transform of temperature series
# fft (from https://stackoverflow.com/questions/48622933/python-performing-fft-on-csv-values-using-scipy-documentation)
import numpy as np
from numpy.fft import rfft, rfftfreq
import matplotlib.pyplot as plt

temperatureSeries = sensorLog.filter(items=['Temp']).values.flatten() #numpy array format
dt = 15/(24*60) #time increment in unit of days (every 15 minutes)

fft = rfft(temperatureSeries)*dt
freq = rfftfreq(len(temperatureSeries),d=dt)

plt.figure(figsize=(20,5))
plt.xlim([0.1,5])
plt.ylim([0, 2000])
plt.xlabel('[Period in Days]')
plt.title(f'Temperature Frequencies (Fourier Transform) / {UpdatedTimestamp}')
axs = plt.bar(freq, abs(fft), width=0.025)
plt.savefig('FFT-Temperature')

In [ ]:
# Calculate and plot Fourier transform of temperature series

pressureSeries = sensorLog.filter(items=['Pressure']).values.flatten() #numpy array format

fft = rfft(pressureSeries)*dt
freq = rfftfreq(len(pressureSeries),d=dt)

plt.figure(figsize=(20,5))
plt.xlim([0.01, 10.1])
plt.ylim([0, 30000])
plt.xlabel('[Period in Days]')
plt.title(f'Pressure Frequencies (Fourier Transform) / {UpdatedTimestamp}')
axs = plt.bar(freq, abs(fft), width=0.025)
plt.savefig('FFT-Pressure')

In [ ]:
freq

In [ ]:
# ftp plots to website

from AlvaziFTP import AlvaziFTP
ftpSession = AlvaziFTP()

png_files = glob.glob('*.png')
for png_file in png_files:
    ftpSession.uploadBinary(png_file)
    print( png_file )

ftpSession.quit()

In [ ]:
# Get index of minimum and maximum temperature overall
#https://www.geeksforgeeks.org/get-the-index-of-maximum-value-in-dataframe-column/

print(sensorLog[['Temp']].idxmin(), sensorLog[['Temp']].idxmax())

In [ ]:
sensorLog.loc[today:today][['Temp']].idxmin()

In [ ]:
# create dataframe with time of minimum and maximum temperature by date
# adapted from https://stackoverflow.com/questions/29569365/how-to-obtain-the-distribution-of-idmax-idmin-in-timeseries-data
sensorLogByDate = sensorLog.groupby(sensorLog.index.date)
timeMinMaxTemp = pd.DataFrame({'MinTemp_Time': sensorLogByDate['Temp'].idxmin().dt.time, 
                               'MaxTemp_Time': sensorLogByDate['Temp'].idxmax().dt.time})

In [ ]:
timeMinMaxTemp.tail(10)

In [ ]:
# plot the min/max temperature times by date
minTemp_Time = pd.to_datetime(timeMinMaxTemp.MinTemp_Time, format='%H:%M:%S') #.dt.time
maxTemp_Time = pd.to_datetime(timeMinMaxTemp.MaxTemp_Time, format='%H:%M:%S') #.dt.time
plt.figure(figsize=(20,8))
plt.plot(timeMinMaxTemp.index, minTemp_Time, marker='*', linestyle='None')
plt.plot(timeMinMaxTemp.index, maxTemp_Time, marker='+', linestyle='None')
plt.grid()

https://stackoverflow.com/questions/58263983/how-to-plot-date-in-x-axis-time-in-y-axis-with-pandas-matplotlib-and-present-ti

(add also sunrise sunset times)

In [ ]:
minTemp_Time

In [ ]:
sensorLog.tail(50)

In [ ]:
# Calculate difference between successive timestamps and print those with absolute value > 15 minutes
# This should show power outages or other downtimes
diffs = sensorLog.index.to_series().diff() #.dropna()
diffs_in_seconds = diffs.dt.total_seconds()
gap_Minutes = 15
filtered_diffs = [(str(idx), diff) for idx, diff in zip(diffs.index[1:], diffs_in_seconds) if abs(diff) > gap_Minutes*60]
print(f"Gaps in time series greater than {gap_Minutes} minutes:")
for timestamp, diff in filtered_diffs:
    print(f"Timestamp: {timestamp}, Difference: {diff/60} minutes")